Collate the data needed to calculate THQ

整理计算THQ需要的数据

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_country_csv = "C:/Users/dell/OneDrive/file/csv/"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"


path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"


path_3_sw_forecast = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"

path_4_risk_assess = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part4_assess/"
path_4_risk_assess_grid = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part4_assess/grid/"

path_1_describe_fig_fish = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part1_describe/fish_diff/"
path_1_describe_global_map = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part1_describe/global_map/"

mark_num = "25"
meta_name = "meta_data.csv"

list_pfas =['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA', 'PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_pfas_lc = ['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA']
list_pfas_sc = ['PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_color = ["#4d8cbf", "#4f9c8b", "#555c6c", "#d77563", "#7d84a8", "#84aeb8", "#c3473b", "#89756d","#ffb3cc","#9a7ebf","#ffddb8", "#c4eaff", "#d1c6ff", "#c2ffbf", "#f5f5b0",]

### Data Collate

In [ ]:
# weight
df_country = pd.read_excel(path_country_csv + "hi_raw.xlsx",sheet_name='country')
dic_code = df_country.set_index("country_ename")["country_id"].to_dict()

df_weight = pd.read_excel(path_country_csv + "hi_raw.xlsx",sheet_name='bmi')
df_weight["country_id"] = df_weight["country_ename"].map(dic_code)
df_weight = df_weight[['country_id','weight']]

weight_mean = df_weight['weight'].mean()

all_country_ids = df_country['country_id'].unique()
existing_country_ids = df_weight['country_id'].unique()

missing_country_ids = set(all_country_ids) - set(existing_country_ids)

missing_data = pd.DataFrame({'country_id': list(missing_country_ids), 'weight': weight_mean})
df_weight = pd.concat([df_weight, missing_data])
df_weight = df_weight[(df_weight['country_id'].notna())&(df_weight['country_id']!=0)]

df_weight.to_csv(path_4_risk_assess + 'hi/weight.csv',index=False)

In [ ]:
# life_ex
df_life = pd.read_excel(path_country_csv + "hi_raw.xlsx",sheet_name='life')
df_life = df_life[(df_life['Year']>1999)&(df_life['Year']<2021)]

df_avg_life = df_life[['country_ename', "life_ex", 'Year']]
df_avg_life = df_avg_life.rename(columns={'Year':'year'})
df_country = pd.read_excel(path_country_csv + "hi_raw.xlsx",sheet_name='country')
dic_code = df_country.set_index("country_ename")["country_id"].to_dict()

df_avg_life["country_id"] = df_avg_life["country_ename"].map(dic_code)
df_avg_life = df_avg_life[['country_id','life_ex', 'year']]

all_country_ids = df_country['country_id'].unique()
existing_country_ids = df_avg_life['country_id'].unique()
missing_country_ids = set(all_country_ids) - set(existing_country_ids)
all_years = df_avg_life['year'].unique()
yearly_life_mean = df_avg_life.groupby('year')['life_ex'].mean()
missing_data = []
for country_id in missing_country_ids:
    for year in all_years:
        missing_data.append({
            'country_id': country_id, 
            'life_ex': yearly_life_mean[year], 
            'year': year
        })

missing_df = pd.DataFrame(missing_data)
df_avg_life = pd.concat([df_avg_life, missing_df])

df_avg_life = df_avg_life[(df_avg_life['country_id'].notna())&(df_avg_life['country_id']!=0)]
df_avg_life.to_csv(path_4_risk_assess + 'hi/life_ex.csv',index=False)

In [ ]:
# consume
# 海鱼和淡水鱼
# Sea fish and freshwater fish
df_country = pd.read_excel(path_country_csv + "hi_raw.xlsx",sheet_name='country')
dic_code = df_country.set_index("country_ename")["country_id"].to_dict()

df_consume = pd.read_excel(path_country_csv + "hi_raw.xlsx",sheet_name='consume')
df_consume = df_consume[(df_consume['Year Code']>1999)&(df_consume['Year Code']<2021)]
df_consume = df_consume.rename(columns={'Year Code':'year'})

df_consume_sf = df_consume[df_consume['Item']=='Marine Fish, Other']
df_consume_ff = df_consume[df_consume['Item']=='Freshwater Fish']

df_consume_sf["country_id"] = df_consume_sf["country_ename"].map(dic_code)
df_consume_sf_avg = df_consume_sf[['country_id','consume','year']]
yearly_sf_mean = df_consume_sf_avg.groupby('year')['consume'].mean()

all_country_ids = df_country['country_id'].unique()
all_years = df_consume_sf_avg['year'].unique()
all_combinations = pd.DataFrame([(country, year) for country in all_country_ids for year in all_years], 
                                columns=['country_id', 'year'])
df_consume_sf_avg = pd.merge(all_combinations, df_consume_sf_avg, on=['country_id', 'year'], how='left')

df_consume_sf_avg.to_csv(path_4_risk_assess + 'hi/sf_consume.csv',index=False)

df_consume_ff["country_id"] = df_consume_ff["country_ename"].map(dic_code)
df_consume_ff_avg = df_consume_ff[['country_id','consume','year']]

yearly_ff_mean = df_consume_ff_avg.groupby('year')['consume'].mean()
df_consume_ff_avg = pd.merge(all_combinations, df_consume_ff_avg, on=['country_id', 'year'], how='left')
df_consume_ff_avg.to_csv(path_4_risk_assess + 'hi/ff_consume.csv',index=False)

### Organize grid data

In [ ]:
df_country_id = pd.read_csv(path_4_risk_assess_grid + 'country_id.csv')
df_country_id = df_country_id[df_country_id['country_id']!=0]
df_coast = pd.read_csv(path_4_risk_assess_grid + 'coast_distance.csv')

df_type = pd.read_csv(path_4_risk_assess_grid + 'type.csv')
df_merged = pd.merge(df_country_id, df_type, on=['lon', 'lat'], how='left')

df_merged = pd.merge(df_merged, df_coast, on=['lon', 'lat'], how='left')
pollution_nan = df_merged['population'].isna().sum()
print(f"Number of unmatched pollution data: {pollution_nan}")
type_nan = df_merged['type'].isna().sum()
print(f"Number of unmatched type data: {type_nan}")
type_nan = df_merged['coast_distance'].isna().sum()
print(f"Number of unmatched type data: {type_nan}")

print(df_merged.columns)
df_merged.to_csv(path_4_risk_assess_grid + 'grid.csv',index=False)

In [ ]:
df_type = pd.read_csv(path_4_risk_assess_grid + 'type.csv')
df_sw = pd.read_csv(path_4_risk_assess + 'pfas/sw_concentration.csv')
df_sw = df_sw.rename(columns={'lon_grid':'lon','lat_grid':'lat'})

df_grid = pd.read_csv(path_4_risk_assess_grid + 'grid.csv')

df_sw_type = pd.merge(df_sw, df_type, on=['lon', 'lat'], how='left')
df_sw_type = df_sw_type.rename(columns={'type':'forecast_type'})
df_sw_type = df_sw_type[['lon','lat','forecast_type']]
df_grid_final = pd.merge(df_grid, df_sw_type, on=['lon', 'lat'], how='left')

df_grid_final.to_csv(path_4_risk_assess_grid + 'new_grid.csv',index=False)

### Nearest grid

In [ ]:
import pandas as pd
import numpy as np


def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # haversine
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    return c * r


def get_nearest_grid(df_need, df_search):
    nearest_grid_lon = []
    nearest_grid_lat = []

    for idx, row in df_need.iterrows():
        lat, lon, country_id = row['lat'], row['lon'], row['country_id']
        df_search['distance'] = haversine(lat, lon, df_search['lat'], df_search['lon'])
        grid1_same_country = df_search[df_search['country_id'] == country_id].nsmallest(1, 'distance')
        if grid1_same_country.empty:
            grid1_diff_country = df_search[df_search['country_id'] != country_id].nsmallest(1, 'distance')
            nearest_grid_lon.append(grid1_diff_country.iloc[0]['lon'])
            nearest_grid_lat.append(grid1_diff_country.iloc[0]['lat'])
            
        else:
            nearest_grid_lon.append(grid1_same_country.iloc[0]['lon'])
            nearest_grid_lat.append(grid1_same_country.iloc[0]['lat'])
    return nearest_grid_lon, nearest_grid_lat

In [ ]:
df_grid = pd.read_csv(path_4_risk_assess_grid + 'new_grid.csv')
df_grid_need = df_grid[(df_grid['population'].notna()) & (df_grid['population']>0)]
print(df_grid.shape)
print(df_grid_need.shape)

df_need1 = df_grid_need.copy()
print(df_need1.shape)
df_need1 = df_need1[['lon','lat','country_id']]

df_search1 = df_grid[(df_grid['type']==1)&(df_grid['forecast_type'].notna())]
print(df_search1.shape)
df_search1 = df_search1[['lon','lat','country_id']]

df_need2 = df_grid_need.copy()
print(df_need2.shape)
df_need2 = df_need2[['lon','lat','country_id']]

df_search2 = df_grid[(df_grid['type']==0)&(df_grid['forecast_type'].notna())]
print(df_search2.shape)
df_search2 = df_search2[['lon','lat','country_id']]


In [ ]:
sf_nearest_grid_lon, sf_nearest_grid_lat = get_nearest_grid(df_need1, df_search1)
df_need1['sf_nearest_lon'] = sf_nearest_grid_lon
df_need1['sf_nearest_lat'] = sf_nearest_grid_lat

df_need1 = df_need1[['lon','lat','sf_nearest_lon', 'sf_nearest_lat']]

ff_nearest_grid_lon, ff_nearest_grid_lat = get_nearest_grid(df_need2, df_search2)
df_need2['ff_nearest_lon'] = ff_nearest_grid_lon
df_need2['ff_nearest_lat'] = ff_nearest_grid_lat

df_need2 = df_need2[['lon','lat','ff_nearest_lon', 'ff_nearest_lat']]

df_neareat = pd.merge(df_need1, df_need2, on=['lon', 'lat'], how='left')
df_neareat.to_csv(path_4_risk_assess_grid + 'nearest_grid.csv',index=False)

### Grid consumption

In [ ]:
import pandas as pd

df_grid_pop = pd.read_csv(path_4_risk_assess_grid + 'new_grid.csv')
df_grid_pop = df_grid_pop[['lon', 'lat', 'country_id', 'type', 'coast_distance', 'forecast_type']]
df_pop_year = pd.read_csv(path_4_risk_assess_grid + 'population_year.csv')
merged_df = pd.merge(df_pop_year, df_grid_pop, on=['lon', 'lat'], how='left')
merged_df.to_csv(path_4_risk_assess_grid + 'new_grid_year.csv',index=False)
pop_summary = merged_df.groupby(['country_id', 'year'])['pop'].sum().reset_index()
pop_summary.to_csv(path_4_risk_assess + '/hi/country_pop.csv',index=False)


In [ ]:
import pandas as pd
import numpy as np

df_country = pd.read_csv(path_4_risk_assess + '/hi/country_pop.csv')
df_country = df_country[df_country['pop'] > 0]

df_ff_consume = pd.read_csv(path_4_risk_assess + '/hi/ff_consume.csv')
df_ff_consume['consume'] = df_ff_consume['consume'] / 365
df_ff_consume = df_ff_consume.rename(columns={'consume': 'ff_consume'})

df_sf_consume = pd.read_csv(path_4_risk_assess + '/hi/sf_consume.csv')
df_sf_consume['consume'] = df_sf_consume['consume'] / 365
df_sf_consume = df_sf_consume.rename(columns={'consume': 'sf_consume'})

df_country = pd.merge(df_country, df_ff_consume[['country_id', 'year', 'ff_consume']], 
                      on=['country_id', 'year'], how='left')
df_country = pd.merge(df_country, df_sf_consume[['country_id', 'year', 'sf_consume']], 
                      on=['country_id', 'year'], how='left')


def fill_missing(df, column):
    df[column] = df.groupby('country_id')[column].transform(lambda x: x.fillna(x.median()))
    global_median = df.groupby('year')[column].transform('median')
    df[column] = df[column].fillna(global_median)
    return df

df_country = fill_missing(df_country, 'ff_consume')
df_country = fill_missing(df_country, 'sf_consume')

df_country['ff_consume_sum'] = df_country["ff_consume"] * df_country["pop"]
df_country['sf_consume_sum'] = df_country["sf_consume"] * df_country["pop"]

df_country.to_csv(path_4_risk_assess + '/hi/avg_consume_median.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

def process_year_data(df_grid, df_avg_consume, year):
    df_grid_year = df_grid[df_grid['year'] == year]
    df_avg_consume_year = df_avg_consume[df_avg_consume['year'] == year]
    
    df_grid_inland = df_grid_year[(df_grid_year['type']==0) | (df_grid_year['pop'].notna())]

    dict_sf = df_avg_consume_year.set_index("country_id")["sf_consume"].to_dict()
    dict_ff = df_avg_consume_year.set_index("country_id")["ff_consume"].to_dict()

    df_grid_inland['sf_consume'] = df_grid_inland["country_id"].map(dict_sf)
    df_grid_inland['ff_consume'] = df_grid_inland["country_id"].map(dict_ff)

    df_grid_inland['ad_fa1_num'] = 0.22222 * np.exp(-0.002 * df_grid_inland['coast_distance'])
    
    max_val = df_grid_inland['ad_fa1_num'].max()
    min_val = df_grid_inland['ad_fa1_num'].min()
    list_id = df_grid_inland['country_id'].unique().tolist()
    
    df_grid_inland['ad_fa_per_sf'] = (df_grid_inland['ad_fa1_num'] - min_val) / (max_val - min_val)
    
    for country_id in list_id:
        type_0_count = df_grid_inland[(df_grid_inland['country_id'] == country_id) & (df_grid_inland['type'] == 0)].shape[0]
        
        if type_0_count == 1:
            df_grid_inland.loc[(df_grid_inland['country_id'] == country_id), 'ad_fa_per_ff'] = 1
        else:
            df_grid_inland.loc[(df_grid_inland['country_id'] == country_id), 'ad_fa_per_ff'] = 1 - df_grid_inland.loc[(df_grid_inland['country_id'] == country_id), 'ad_fa_per_sf']

    df_grid_inland['re1_sf_consume'] = df_grid_inland['sf_consume'] * df_grid_inland['ad_fa_per_sf']
    df_grid_inland['re1_ff_consume'] = df_grid_inland['ff_consume'] * df_grid_inland['ad_fa_per_ff']

    df_grid_inland['re1_sf_consume_sum'] = df_grid_inland['re1_sf_consume'] * df_grid_inland['pop']
    df_grid_inland['re1_ff_consume_sum'] = df_grid_inland['re1_ff_consume'] * df_grid_inland['pop']

    sf_consume_sum_dict = df_grid_inland.groupby('country_id')['re1_sf_consume_sum'].sum().to_dict()
    ff_consume_sum_dict = df_grid_inland.groupby('country_id')['re1_ff_consume_sum'].sum().to_dict()

    df_avg_consume_year['re1_sf_consume_sum'] = df_avg_consume_year['country_id'].map(sf_consume_sum_dict)
    df_avg_consume_year['re1_ff_consume_sum'] = df_avg_consume_year['country_id'].map(ff_consume_sum_dict)

    df_avg_consume_year['tf_ff'] = df_avg_consume_year['ff_consume_sum'] / df_avg_consume_year['re1_ff_consume_sum']
    df_avg_consume_year['tf_ff'].fillna(1, inplace=True)

    df_avg_consume_year['tf_sf'] = df_avg_consume_year['sf_consume_sum'] / df_avg_consume_year['re1_sf_consume_sum']
    df_avg_consume_year['tf_sf'].fillna(1, inplace=True)

    dict_sf_fa = df_avg_consume_year.set_index("country_id")["tf_sf"].to_dict()
    dict_ff_fa = df_avg_consume_year.set_index("country_id")["tf_ff"].to_dict()

    df_grid_inland['tf_sf'] = df_grid_inland['country_id'].map(dict_sf_fa)
    df_grid_inland['tf_ff'] = df_grid_inland['country_id'].map(dict_ff_fa)

    df_grid_inland['re2_sf_consume'] = df_grid_inland['re1_sf_consume'] * df_grid_inland['tf_sf']
    df_grid_inland['re2_ff_consume'] = df_grid_inland['re1_ff_consume'] * df_grid_inland['tf_ff']

    df_grid_inland_output = df_grid_inland[['lon','lat','country_id', 'type','re2_sf_consume','re2_ff_consume','pop','coast_distance', 'year']]
    df_grid_inland_output = df_grid_inland_output.rename(columns={'re2_sf_consume':'sf_consume','re2_ff_consume':'ff_consume'})
    df_grid_inland_output = df_grid_inland_output[df_grid_inland_output['pop']>0]
    
    return df_grid_inland_output

df_grid = pd.read_csv(path_4_risk_assess_grid + 'new_grid_year.csv')
df_avg_consume = pd.read_csv(path_4_risk_assess + '/hi/avg_consume_median.csv')

years = df_grid['year'].unique()

results = []
for year in years:
    result = process_year_data(df_grid, df_avg_consume, year)
    results.append(result)

final_result = pd.concat(results, ignore_index=True)

final_result.to_csv(path_4_risk_assess_grid + 'consume_grid_year.csv', index=False)

### Grid PFAS concentration

In [ ]:
import pandas as pd
df_grid = pd.read_csv(path_4_risk_assess_grid + 'consume_grid_year.csv')

df_nearest = pd.read_csv(path_4_risk_assess_grid + 'nearest_grid.csv')

df_grid = df_grid.merge(df_nearest, on=['lon', 'lat'], how='left')

path_5_sw = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/s7_rf_output/only_pfas/"
path_5_lr = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/lr_forecast/s7_rf_output/only_pfas/"

for str_pfas in ['FOSA', 'PFBA',
       'PFBS', 'PFDA', 'PFDoDA', 'PFHpA', 'PFHxA', 'PFHxS', 'PFNA', 'PFOA',
       'PFOS', 'PFPeA', 'PFTeDA', 'PFTrDA', 'PFUnDA']:
   df_lr = pd.read_csv(path_5_lr + 'lr_'+str_pfas+'.csv')
   df_sw = pd.read_csv(path_5_sw + 'sw_'+str_pfas+'.csv')

   for df_select in [df_lr, df_sw]:
      if df_select is df_lr:
         str_marke = 'lr'
         df_data = df_lr.copy()
      else:
         str_marke = 'sw'
         df_data = df_sw.copy()
      print(str_marke)
      df_grid_pfas = df_grid.copy()
      print(df_grid_pfas.columns)
      list_remain = ['lon','lat','year','country_id','ff_consume','sf_consume',str_pfas + '_min',str_pfas + '_max',str_pfas + '_median']
      df_grid_pfas['sw_consume'] = 2*0.2*0.5

      df_grid_pfas = df_grid_pfas.merge(df_data, left_on=['ff_nearest_lon', 'ff_nearest_lat', 'year'], right_on=['lon_grid', 'lat_grid', 'year'], how='left')
      df_grid_pfas = df_grid_pfas[list_remain]
      df_grid_pfas.to_csv(path_4_risk_assess + 'pfas/' +str_marke + '_' + str_pfas+'.csv', index=False)